<a href="https://colab.research.google.com/github/dbuendiab/notebook-prueba/blob/master/SQL_B%C3%A1sico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción al SQL

Este cuaderno interactivo te permite aprender y practicar la sintaxis SQL de una forma espero que sencilla.

Para tener una base con la que practicar, he bajado una tabla de datos sobre contagios de COVID en Cataluña, obtenida en el portal  [Dades Obertas de la Generalitat](https://analisi.transparenciacatalunya.cat/Salut/Registre-de-casos-de-COVID-19-realitzats-a-Catalun/jj6z-iyrp/data).

Pero, antes de seguir, ejecuta las celdas ocultas de la siguiente sección, que prepara el cuaderno para poder seguir las prácticas.

## Preparativos

Ejecuta las tres celdas siguientes:
* **Montar Google Drive**: para poder acceder a la base de datos
* **Conexión a base de datos (COVID)**: para establecer la conexión a la base de datos
* **Creación del comando %%sql**: para escribir código SQL directamente.

Una vez hecho, ejecuta la **celda de prueba** a continuación.

In [ ]:
#@title Montar Google Drive
#@markdown Ejecutar para acceder a la base de datos
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
!ls MyDrive/


'20141203 Plan Estrategico PS.xls'
'20141203 Plan Estrategico PS.xls.gsheet'
'20150323 MWC2015_empresas_emails_www.xls'
'20150323 MWC2015_empresas_emails_www.xls.gsheet'
'2020-08 (1).csv'
'2020-08 (2).csv'
 2020-08.csv
 2020-08.gsheet
'Abono social  papá.gdoc'
' AGUSTINA SALVADOR.gdoc'
 Amigotes.jpg
'AngularJS - Phonecat - Getting Started.gdoc'
'AngularJS - Step 0.gdoc'
'Aniversario Alba - Intervención.gdoc'
'Arrels: el grupo.gdoc'
'Asuntos Ignasi'
 ATAM.gdoc
'Autoedición - Publicar libro uno mismo.gdoc'
'AWS - Montar daemon Quijote.gdoc'
'AWS: nuevo usuario para acceso SSH.gdoc'
 B
'Birdy - People Help The People - YouTube.png'
 Blog_ASP_Net.rar
'Blogs (enlaces).gdoc'
'Blues trigonométrico.mp3'
'borrador dolores motos salvador.gdoc'
'BORRADOR ESPERANZA LUNA .docx'
'BORRADOR LOPEZ PEREZ OCTUBRE 2017.gdoc'
'BORRADOR SRA GOMEZ 2.doc'
'Capítulos largos y cortos.gdoc'
'Carta de recomendación Abril Díaz.gdoc'
'Casandra: No te preocupes por tu hijo.gdoc'
'Casas de la familia.gmap'
'Ch

In [ ]:
#@title Conexión a base de datos (COVID)
#@markdown Ejecutar para seguir con el tutorial
import sqlite3
cn = sqlite3.connect("/gdrive/MyDrive/covid.db")
cr = cn.cursor()
print("Conexión establecida")

Conexión establecida


In [ ]:
#@title Creación del comando %%sql

#@markdown Normalmente en las celdas escribes código Python, pero los códigos mágicos (%%comando) permiten usar otros lenguajes.

#@markdown Aquí crearemos el comando **%%sql**, que te permitirá escribir comandos SQL directamente en las celdas siguientes, usando un formato como el del siguiente ejemplo:

#@markdown ```%%sql```<br>
#@markdown ```SELECT * FROM data LIMIT 10```

from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)
import json
import prettytable

@register_cell_magic
def sql(line, cell):
    parametros = line.split(' ')

    ## Habilito por defecto la visualización en bonito
    ## Si quieres recuperar el conjunto de datos, hay que 
    ## poner 'datos' en la línea del comando mágico, como en
    ##
    ## %%sql datos

    imprime = True
    if 'datos' in parametros:
      imprime = False
    ## imprime = 'tabla' in parametros

    ## Para alinear las celdas de la tabla a la izquierda
    leftalign = 'lalign' in parametros
    data = {}
    
    cr.execute(cell)
    if cr:
        if imprime:
            x = prettytable.from_db_cursor(cr)
            if leftalign:
                for fld in x.field_names:
                    x.align[fld] = 'l'
            print(x)
        else:
            return cr.fetchall()

# Celda de prueba

 El resultado debe ser 256360, que son las filas que contiene la tabla.

In [ ]:
%%sql
SELECT COUNT(*) FROM data

+----------+
| COUNT(*) |
+----------+
|  256360  |
+----------+


# Lo básico de las consultas SQL

Una frase SQL tiene esta estructura básica:

```
SELECT (lista campos y expresiones)
FROM (tabla o tablas)
WHERE (expresión lógica)
GROUP BY (lista campos y expresiones)
HAVING (expresión lógica)
ORDER BY (lista campos para la ordenación)
```
Cada uno de estos 6 elementos es una cláusula principal del lenguaje, que puede
complicarse tanto como sea necesario, pero siempre conservando este orden. Las cláusulas ```WHERE```, ```GROUP BY```, ```HAVING``` y ```ORDER BY``` son opcionales.

Es importante entender el orden en que un motor SQL ejecuta estas cláusulas, ya que no corresponde con el que aparece en la sintaxis. El verdadero orden de ejecución es el que sigue:

* **FROM**: Lo primero es saber dónde están los datos.
* **WHERE**: Decidir qué selección de filas necesitamos (opcional; puede que las queramos todas).
* **GROUP BY**: Para un determinado tipo de consultas, lo que queremos es hacer montoncitos por categorías, para obtener datos para cada montoncito. Ya lo veremos, pero esta cláusula es la que hace esos montoncitos (opcional; las consultas normales no necesitan montoncitos).
* **HAVING**: Si hemos hecho montoncitos, esta cláusula es un WHERE sobre cosas que hayamos calculado en esos montoncitos. Es opcional también.
* **SELECT**: Esta cláusula es obligatoria. Aquí tendremos una lista de los campos o expresiones que vamos a devolver al usuario (que provienen de todo lo que hemos calculado previamente).
* **ORDER BY**: Una última cosa opcional es ordenar la tabla de resultados de una u otra manera.

Veremos las distintas cláusulas más adelante, pero primero vamos con una cosa importante que no se explica normalmente.



# ¿Qué datos tiene mi tabla?

Para hacer consultas a una base de datos, primero hay que saber de qué datos estamos hablando.

Hay una serie de comandos SQL digamos *prospectivos* que permiten hacerse una idea del contenido de una tabla, el problema es que si aún no sabemos SQL no vamos a entenderlos completamente. Aún así, son tan útiles que los voy a explicar, y así vamos viendo qué datos tenemos para hacer nuestras pruebas (y, de paso, iremos aprendiendo SQL).



## ¿De cuántas filas hablamos?

No es lo mismo una tabla con 100 filas que otra con 100.000.000. Si pedimos **todas** las filas de la tabla, en el segundo caso posiblemente no lleguemos a ver el resultado, porque el servidor se atascará.

Así que la primera consulta que haremos será esta (que ya vimos antes):

In [ ]:
%%sql 
SELECT COUNT(*) 
FROM data

+----------+
| COUNT(*) |
+----------+
|  256360  |
+----------+


### Explicación

**FROM data** significa que seleccionamos la tabla **data** para nuestra consulta.

**SELECT COUNT(*)** significa *'mira cuántas filas hay y me lo dices'*.

**SELECT** es la cláusula de selección, donde podemos poner una lista de campos o expresiones que queremos obtener de la consulta.

**COUNT(*)** es una función de agregación. Normalmente se usa con los montoncitos que genera la cláusula **GROUP BY**, pero si no tenemos esta cláusula, se entiende que el montoncito es único e incluye todas las filas de la tabla.

El asterisco **(*)** viene a significar **TODAS** las filas, pero no vamos a profundizar más aquí. Baste decir que la función COUNT() puede contar de varias maneras distintas, pero la más corriente es esta. 

## ¿Qué columnas contiene?

Aquí se trata de saber tanto el nombre de las columnas como una muestra de su contenido. Para ello nos basta (en principio) una pequeña muestra. Vamos con el SQL:

In [ ]:
%%sql

SELECT * 
FROM data 
LIMIT 10

+--------------+-------------+--------------+----------+--------------------+----------+
| TipusCasData | ComarcaCodi | MunicipiCodi | SexeCodi | TipusCasDescripcio | NumCasos |
+--------------+-------------+--------------+----------+--------------------+----------+
|  2020-11-27  |      19     |    17114     |    0     |      Sospitós      |   135    |
|  2020-11-27  |      09     |    43104     |    1     |      Sospitós      |    1     |
|  2020-11-27  |      03     |    08013     |    0     |      Sospitós      |    1     |
|  2020-11-27  |      07     |    08218     |    0     |      Sospitós      |    2     |
|  2020-11-27  |      18     |    25058     |    0     |      Sospitós      |    3     |
|  2020-11-27  |      11     |    08211     |    0     |      Sospitós      |    12    |
|  2020-11-27  |      11     |    08001     |    0     |      Sospitós      |    4     |
|  2020-11-27  |      28     |    17137     |    0     |      Sospitós      |    8     |
|  2020-11-27  |     

### Explicación

Vamos a señalar solo las cláusulas que incluyan novedades:

* **LIMIT 10**: esto significa que solo queremos recuperar 10 filas. Total, queremos una degustación, tan solo tener una idea de lo que hay. Esta es una cláusula que no pusimos entre las básicas, la razón es que no es un estándar, cada fabricante usa una sintaxis. Esta es la que usa **sqlite**, la base de datos que estoy usando para este cuaderno. En el **SQL Server de Microsoft**, el comando sería ```SELECT TOP 10 ...```. No sé ahora si otros fabricantes tienen otras sintaxis para esto.

* **SELECT ***: El asterisco aquí significa *'enséñame todas las columnas que haya en la tabla'*. Esta es la sintaxis más común para la cláusula **SELECT**. De hecho, la versión equivalente sería:

```
SELECT TipusCasData, ComarcaCodi, MunicipiCodi, 
       SexeCodi, TipusCasDescripcio, NumCasos
FROM data
LIMIT 10
```

lo que pone en evidencia el ahorro que conlleva usar el asterisco.

## ¿Qué contiene cada columna?

Esta tabla tiene la mayoría de sus campos codificados, así que es difícil a simple vista saber qué significa su contenido. Sin embargo, la cuestión es importante para entender los datos, así que tenemos que investigar un poco las columnas en concreto.

De la muestra anterior, ya vemos que *TipusCasData* parece contener fechas: una primera prospección sería saber de qué rango de fechas hablamos, es decir, cuál es la mayor y cuál la menor. Para ello, tenemos las funciones de agregación **MIN(*campo*)** y **MAX(*campo*)**.

Mira de construir la consulta correspondiente...

In [ ]:
%%sql

SELECT <rellena esto!>
FROM data

### Solución

In [ ]:
%%sql

SELECT MIN(TipusCasData), MAX(TipusCasData)
FROM data

+-------------------+-------------------+
| MIN(TipusCasData) | MAX(TipusCasData) |
+-------------------+-------------------+
|     2020-02-26    |     2020-11-27    |
+-------------------+-------------------+


#### Explicación

Ya sabemos algo importante de nuestra tabla: que contiene datos que van desde febrero hasta noviembre.

Las funciones **MIN(), MAX()** son funciones de agregado, como **COUNT()** y, por tanto, funcionarían con los montoncitos generados por la cláusula **GROUP BY**, si existiera. Como dijimos, si no existe el motor de la base de datos entiende que toda la tabla es un mismo montoncito.

Por esa misma razón, en esta consulta solo sale una fila de resultados: solo tenemos un montoncito, así que solo puede haber un valor máximo y mínimo del conjunto completo.

## ¿Cuántos valores distintos tiene una columna?

Las columnas pueden contener datos muy variados. 

Puedes tener un mismo valor que se repite a menudo en las distintas filas (como, por ejemplo, el código de comarca), o puede ser que sea muy variado, como las cantidades de cada producto en un almacén.

Por tanto, nos interesa distinguir los campos de código de los que contienen solo datos. Ya sabemos que tenemos más de 200.000 filas, ahora querríamos saber cuántos valores distintos tiene cada columna (al menos, en aquellas columnas que nos parecen mejores candidatas a ser columnas de códigos o *dimensiones*, como las llaman a veces).

Veamos la consulta para esto (puedes probar a ejecutar con distintos campos y ver qué valores obtienes):

In [ ]:
%%sql

SELECT COUNT(DISTINCT sexeCodi)
FROM data

+--------------------------+
| COUNT(DISTINCT sexeCodi) |
+--------------------------+
|            2             |
+--------------------------+


### Explicación

**COUNT(DISTINCT expresion)**: Esta es una de las variaciones que tiene la función **COUNT()**. Lo que significa el **DISTINCT** es *'calcula para cada fila la expresión dada, y cuenta cuántos valores distintos se obtienen'*. *Expresion* a menudo es el nombre de la columna, pero podría ser cualquier expresión más complicada (por ejemplo, contar cuántos valores de la columna X empiezan por la misma letra).

## ¿Y qué valores en concreto tiene una columna?

Para aquellas columnas con pinta de códigos, podemos obtener una lista de valores de una forma muy simple. Tomemos, por ejempo, la columna *TipusCasDescripcio*, un caso interesante porque tiene pocos valores y, además, no vienen codificados.

In [ ]:
%%sql lalign  ## lalign no es más que un indicador para que alinee los resultados a la izquierda

SELECT DISTINCT TipusCasDescripcio
FROM data
LIMIT 100

+------------------------+
| TipusCasDescripcio     |
+------------------------+
| Sospitós               |
| Positiu PCR            |
| Positiu per ELISA      |
| PCR probable           |
| Positiu per Test Ràpid |
| Epidemiològic          |
+------------------------+


### Explicación

* **SELECT DISTINCT**: Devuelve la colección de filas encontradas, pero descartando las que sean iguales. En las columnas que corresponden a dimensiones, estos valores son limitados, así que podemos hacer un **SELECT** sin miedo a devolver miles de filas. Sin embargo, siempre que hacemos prospección es bueno cubrirse con una cláusula **LIMIT n** por si las moscas.

# Otras tablas

Hemos visto que tenemos columnas con códigos (que no sabemos en principio qué significan) y otra (*TipusCasDescripcio*) que viene con códigos legibles.

En una base de datos suele hacerse un proceso que se llama normalización, y que consiste en crear tablas con los códigos y descripciones correspondientes a una columna de la tabla principal, donde solo vienen los códigos. El SQL permite hacer una consulta que combine los datos de todas esas tablas.

Pero primero hay que saber qué tablas son. 

In [ ]:
%%sql 

SELECT name
FROM sqlite_master 
WHERE type = 'table' AND name != 'data'

+-----------+
|    name   |
+-----------+
| Municipis |
|  Comarcas |
|    Sexe   |
|    data   |
+-----------+


### Explicación

* **WHERE ... name != 'data'**: comento esto - se trata de que nos muestre todas las tablas menos la que se llama *data*, que ya sabemos que es la principal, con la que hemos estado trabajando hasta ahora. Puedes jugar a cambiar cosas en la consulta para ver qué otras cosas hay en esta tabla de sistema

#### Tablas de sistema

Todo sistema de base de datos guarda información sobre sí misma en tablas que es posible consultar. El único problema es que esas tablas de sistema no están estandarizadas, de modo que cada fabricante las implementa según su criterio.

En el caso de *Sqlite*, tenemos la tabla **sqlite_master**, que hemos usado aquí.

Para *Oracle*, tenemos **all_tables** y **user_tables**, entre otras.

En *SQL Server*, está **SYSOBJECTS**.

En fin, que hay que estudiar cada caso por separado para saber dónde están los metadatos de cada sistema.


### Inspección de tablas asociadas

Ahora puedes dedicar un momento a consultar el contenido de estas tablas. Ten cuidado de ver el número de filas primero, o de mantener una cláusula **LIMIT** para más seguridad.

In [ ]:
%%sql lalign

SELECT * 
FROM Municipis
LIMIT 100

# Usar vistas en SQL

Tener una base de datos normalizada tiene muchas ventajas: se evita repetir información, se reduce el tamaño y se evita que haya información defectuosa (nadie escribirá *Barcelona* en un sitio y *Bcn* en otro, por ejemplo). Sin embargo, tiene un inconveniente serio, y es que las consultas se complicarán, si queremos sacar información legible.

Para ello, SQL permite combinar tablas, algo que veremos más adelante. Lo que veremos ahora es que esas consultas complejas se pueden guardar dándoles un nombre y usarlas como si fueran tablas, lo que es especialmente conveniente. En este caso, tenemos una tabla llamada **vista** que nos ahorrará (por ahora) el tener que saber cómo se combinan las distintas tablas. El resultado:

In [ ]:
%%sql
SELECT *
FROM vista
LIMIT 10

+--------------+-------------+-------------------+--------------+--------------------------+----------+----------------+--------------------+----------+
| TipusCasData | ComarcaCodi | ComarcaDescripcio | MunicipiCodi |    MunicipiDescripcio    | SexeCodi | SexeDescripcio | TipusCasDescripcio | NumCasos |
+--------------+-------------+-------------------+--------------+--------------------------+----------+----------------+--------------------+----------+
|  2020-11-27  |      19     |      Garrotxa     |    17114     |           Olot           |    0     |      Home      |      Sospitós      |   135    |
|  2020-11-27  |      09     |     Baix Ebre     |    43104     |        El Perelló        |    1     |      Dona      |      Sospitós      |    1     |
|  2020-11-27  |      03     |    Alt Penedès    |    08013     |  Avinyonet del Penedès   |    0     |      Home      |      Sospitós      |    1     |
|  2020-11-27  |      07     |       Bages       |    08218     | Sant Joan de Vil

# Las cláusulas SQL en detalle

Veremos ahora las cláusulas SQL, en el orden en que las usa el motor de cualquier base de datos, mirando sus variaciones.

## Cláusula FROM

**FROM tabla**: esta cláusula, como dijimos, es la que inicia la consulta. Su función es seleccionar la tabla (o conjuntos de tablas) de las que vamos a sacar los datos.

La verdadera complejidad de esta cláusula aparecerá cuando se combinen varias tablas, cosa que no vamos a hacer todavía.

De momento nos basta con saber que es el primer paso que hace el motor de datos al ejecutar una consulta SQL: seleccionar la fuente de los datos.



## Cláusula WHERE

**WHERE (expresion lógica)**: la cláusula **WHERE** se ejecuta para cada fila de la tabla seleccionada con **FROM**. La *expresión lógica* puede ser una combinación de operaciones que usualmente combina datos de los campos y devuelve un valor **Verdadero** o **Falso**, en función del cual la fila pasará (o no) este primer filtro de la consulta.

### Ejemplos de expresiones lógicas

#### Operadores de comparación y lógicos

Los típicos operadores que tenemos en matemáticas: &lt;, &gt; =, != (distinto), y los lógicos, que en SQL son **AND, OR y NOT**.
* **numCasos >= 100**: deja pasar las filas que tengan más de 100 en el campo *NumCasos*.
* **numCasos >= 100 AND SexeDescripcio = 'Home'**: deja pasar las filas que cumplan lad dos condiciones: que tengan más de 100 en el campo *NumCasos* y ek valor 'Home' en el campo *SexeDescripció'.
* **TipusCasDescripció = 'Sospitós'**: pasan solo las filas que tengan el valor *'Sospitós'* en el campo *TipusCasDescripció*.

#### BETWEEN: operador de rango

Sirve para filtrar valores que estén entre un valor mínimo y máximo.
* **TipusCasData BETWEEN '2020-10-01' AND '2020-11-01'**: recupera sólo las filas del mes de noviembre.

#### LIKE: operador de comparación

El operador **LIKE** compara el valor de texto de un campo con una determinada plantilla. Los caracteres '%' y '_' se usan como comodines:
* **%** representa a un número variable de caracteres
* **_** representa a un solo carácter

Así, por ejemplo:
* **ComarcaDescripcio LIKE 'S%'**: recupera las filas cuya *'ComarcaDescripcio'* empieza por 'S'.
* **ComarcaDescripcio LIKE '%x%'**: filas que contengan el carácter *'x'* en el campo *'ComarcaDescripcio'*.



In [ ]:
%%sql

SELECT *
FROM Comarcas
WHERE ComarcaDescripcio LIKE '%x%'
LIMIT 10;


+-------------+-------------------+
| ComarcaCodi | ComarcaDescripcio |
+-------------+-------------------+
|      19     |      Garrotxa     |
|      09     |     Baix Ebre     |
|      11     |   Baix Llobregat  |
|      08     |     Baix Camp     |
|      12     |    Baix Penedès   |
|      10     |    Baix Empordà   |
+-------------+-------------------+


### Ejemplo WHERE

Seleccionar las filas que contengan más de 500 casos de *'Positiu PCR'* durante Octubre.

In [ ]:
%%sql

SELECT *
FROM vista
WHERE (tu condición lógica aquí)

### Solución

In [ ]:
%%sql

SELECT * 
FROM vista
WHERE NumCasos > 500 
      AND 
      TipusCasDescripcio = 'Positiu PCR' 
      AND 
      TipusCasData BETWEEN '2020-10-01' AND '2020-11-01'
LIMIT 50;

+--------------+-------------+-------------------+--------------+--------------------+----------+----------------+--------------------+----------+
| TipusCasData | ComarcaCodi | ComarcaDescripcio | MunicipiCodi | MunicipiDescripcio | SexeCodi | SexeDescripcio | TipusCasDescripcio | NumCasos |
+--------------+-------------+-------------------+--------------+--------------------+----------+----------------+--------------------+----------+
|  2020-10-30  |      13     |     Barcelonès    |    08019     |     Barcelona      |    1     |      Dona      |    Positiu PCR     |   605    |
|  2020-10-29  |      13     |     Barcelonès    |    08019     |     Barcelona      |    1     |      Dona      |    Positiu PCR     |   509    |
|  2020-10-28  |      13     |     Barcelonès    |    08019     |     Barcelona      |    1     |      Dona      |    Positiu PCR     |   609    |
|  2020-10-28  |      13     |     Barcelonès    |    08019     |     Barcelona      |    0     |      Home      |    

## Cláusula SELECT

Dejaremos las cláusulas **GROUP BY** y **HAVING** para más adelante, ya que pertenecen a una clase diferente de consulta SQL respecto de las que hemos visto hasta ahora. Hay dos tipos de consultas SQL:

* **Consultas de detalle**: se devuelven todas o parte de las filas originales, ya sean las columnas existentes u otras calculadas *ad hoc*.
* **Consultas de agregación**: se devuelven filas con datos sumarizados, es decir, correspondientes a los distintos grupos creados a partir de las filas originales.

La cláusula **SELECT** se usa para definir qué columnas queremos ver en el resultado final.



## Cláusula ORDER BY



La cláusula **ORDER BY